# This is analysis of the Hotel Reservation Dataset

### 1. First, let's import the necessary files and get info about the various column present in the dataset

In [1]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/DYNABOOK/Desktop/DS_TS_Prac/Hotel Reservations (1).csv")
df.info()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

np.int64(0)

### 2. Now, let's check if there are any null values present within the dataset

In [4]:
df.isnull().sum()

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

### 3. First, let's form the relation between Room type and Avg Price per room

In [15]:
grouped_df = df.groupby("room_type_reserved")["avg_price_per_room"].mean().reset_index()

grouped_df.columns = ["Room Type", "Avg Price"]

fig = px.bar(grouped_df, x="Room Type", y="Avg Price", title="Average Price for Every Room Type")

fig.show()

#### Conclusion : We notice that, Room Type 6 has the highest average price and Room type 3 has the lowest average price.

### 4. Now, let's try and understand how how other factors could potentially be affecting this average price of any given room
  

In [6]:
grouped_df = df.groupby("room_type_reserved")["lead_time"].mean().reset_index()

grouped_df.columns = ["Room Type", "Avg Lead Time"]

fig = px.bar(grouped_df, x="Room Type", y="Avg Lead Time", title="Lead time for every Room Type")

fig.show()

#### Conclusion : We notice that Lead time is inversely proportional to Avg price for every Room type 
##### The rooms with a higher avg price, like Room Types 6 and 7 tend to have a shorter lead time than cheaper Room types like 1 and 2 

### 5. Now, let's find out the best performing room types and try and find out if that's related to the Avg price

In [7]:
grouped_df = df.groupby("room_type_reserved")["booking_status"].count().reset_index()

grouped_df.columns = ["Room Type", "Count of people"]

fig2 = px.bar(grouped_df, x="Room Type", y="Count of people", title="Number of people occupying every Room Type")

fig2.show()

### 6. We notice that Room Type 1, despite not being the cheapest room, is by far the most successful room type. Let's try and find out what reasons could be contributing to that

In [8]:
df_status_pct = df.groupby(["room_type_reserved", "booking_status"]).size().reset_index(name="Count")
df_total = df.groupby("room_type_reserved").size().reset_index(name="Total")

df_status_pct = df_status_pct.merge(df_total, on="room_type_reserved")
df_status_pct["Percentage"] = ((df_status_pct["Count"] / df_status_pct["Total"]) * 100).round(2)

fig4 = px.bar(df_status_pct, 
              x="room_type_reserved", 
              y="Percentage", 
              color="booking_status", 
              title="Booking Status Distribution for Each Room Type (Percentage)", 
              barmode="stack", 
              text="Percentage")

fig4.show()

#### Conclusion : The cancellation rates is same throughout all the room types so the price or performance of any room type isn't directly related with this parameter

### 7. Since, Room Type 1 is more well performing, let's check the mode in which booking is done to find out why people opt for that specific type

In [9]:

room_types = df['room_type_reserved'].unique()
num_rooms = len(room_types)
rows = int(np.ceil(num_rooms / 2))
cols = 2

fig = sp.make_subplots(rows=rows, cols=cols, 
                       specs=[[{'type': 'domain'}, {'type': 'domain'}]] * rows, 
                       subplot_titles=room_types)

row, col = 1, 1

for room in room_types:
    room_df = df[df['room_type_reserved'] == room]
    booking_counts = room_df['market_segment_type'].value_counts(normalize=True).reset_index()
    booking_counts.columns = ['Booking Method', 'Percentage']
    booking_counts["Percentage"] = (booking_counts["Percentage"] * 100).round(2)

    fig.add_trace(go.Pie(labels=booking_counts['Booking Method'], 
                         values=booking_counts['Percentage'], 
                         hole=0.4,
                         textinfo='label+percent',
                         insidetextorientation='horizontal'), 
                  row=row, col=col)

    col += 1
    if col > cols:
        row += 1
        col = 1

fig.update_layout(title_text="Booking Methods for Each Room Type", 
                  showlegend=False, 
                  height=rows * 350, 
                  width=800)

fig.show()



#### Conclusion : We note that the most frequently used booking method for people reserving Room type 1 is through the online portal. But this is the same for all the types!
However, we note that unlike most other rooms, Room type 1 also gets a significant portion of customers from offline bookings. So, people who book rooms via offline methods generally prefer Room type 3 over other types 

### 8. Let's try to find the average price of any particular room based on the method of booking to find out if cheaper rates is why people prefer online booking methods

In [10]:
avg_price_seg = df.groupby("market_segment_type")["avg_price_per_room"].mean().reset_index()

fig = px.bar(avg_price_seg, x="market_segment_type", y="avg_price_per_room", 
             title="Average Price per Room by Market Segment", text_auto=".2f")
fig.show()


#### Conclusion : We note that online prices are the highest and despite that, that's the most used method for booking! This could be an indicator that the web portal for making online bookings is well maintained and people find it easy to use and book rooms!

### 9. Now, let's try to figure out which meal plan is popular among all the customers

In [11]:
meal_pref = df["type_of_meal_plan"].value_counts(normalize=True).reset_index()
meal_pref.columns = ["Meal Plan", "Percentage"]
meal_pref["Percentage"] = (meal_pref["Percentage"] * 100).round(2)

fig = px.bar(meal_pref, x="Meal Plan", y="Percentage", 
             title="Overall Meal Plan Preference", text_auto=True)
fig.show()



#### Conclusion: Meal Type 1 is by far the most popular type of meal plan selected by all customers 

### 10. Comparison of bookings made across the year

In [12]:
bookings_per_month = df.groupby("arrival_month").size().reset_index(name="count")

fig = px.line(bookings_per_month, x="arrival_month", y="count", 
              title="Number of Bookings per Month", markers=True)
fig.show()


#### Conclusion: We noticed that the total bookings peaked during the months of September and October. This could denote some significant event or offer present in these months. 
Now, Let's compare the cancellations done across the year to see if we have draw any insights from it  

In [13]:
cancel_month = df[df["booking_status"] == "Canceled"].groupby("arrival_month").size().reset_index(name="count")

fig = px.line(cancel_month, x="arrival_month", y="count", 
              title="Cancellations per Month", markers=True)
fig.show()


#### As expected, the rate of cancellation is also the highest in these two months with December having the best Booking:Cancelled ratio across all the months

### 11. Let's analyze the behaviour of repeating guests 

In [14]:
repeat_guest_pct = (df["repeated_guest"].sum() / len(df)) * 100

print(f"Percentage of Repeated Guests: {repeat_guest_pct:.2f}%")
repeat_guest_mode = df[df["repeated_guest"] == 1]["market_segment_type"].value_counts(normalize=True).reset_index()
repeat_guest_mode.columns = ["Market Segment", "Percentage"]
repeat_guest_mode["Percentage"] = (repeat_guest_mode["Percentage"] * 100).round(2)

fig = px.bar(repeat_guest_mode, x="Market Segment", y="Percentage", 
             title="Preferred Booking Mode for Repeated Guests", text_auto=True)
fig.show()


Percentage of Repeated Guests: 2.56%


#### We notice that there is scope for improvement in the customer retention policy as only 2.5% of the total customers visit the hotel again for future stays. 
Most of the retained customers are in the form of Corporate employees which could indicate the tie ups that the hotel has with other organizations